In [1]:
import pandas as pd

import tensorflow as tf
from tensorflow import feature_column

from sklearn.model_selection import train_test_split

import warnings 
warnings.filterwarnings('ignore')

In [ ]:
#@title Read data if you are using Google Colab, in case of running this as Jupyter notebook, just omit this cell
from google.colab import files
uploaded = files.upload()

In [3]:
df = pd.read_csv("training.csv")

In [4]:
df.head()

,id,posted_time,updated_time,age,updated_age,app_type,owner_id,owner_type,owner_influence,number_of_likes,number_of_comments,reactivity,is_mentions_me,is_mentions_connections,is_commented_by_me,is_commented_by_connections,is_liked_by_me,is_liked_by_connections,poster_id,poster_gender,poster_is_employee,poster_is_lead,poster_is_in_connections,poster_focus,poster_order,poster_influence,participant1_id,participant1_action,participant1_gender,participant1_is_employee,participant1_is_lead,participant1_is_in_connections,participant1_focus,participant1_order,participant1_influence,participant2_id,participant2_action,participant2_gender,participant2_is_employee,participant2_is_lead,participant2_is_in_connections,participant2_focus,participant2_order,participant2_influence,participant3_id,participant3_action,participant3_gender,participant3_is_employee,participant3_is_lead,participant3_is_in_connections,participant3_focus,participant3_order,participant3_influence,participant4_id,participant4_action,participant4_gender,participant4_is_employee,participant4_is_lead,participant4_is_in_connections,participant4_focus,participant4_order,participant4_influence,participant5_id,participant5_action,participant5_gender,participant5_is_employee,participant5_is_lead,participant5_is_in_connections,participant5_focus,participant5_order,participant5_influence,rank
0,175339,1.514050e+12,1.514050e+12,6.315242e+10,6.315242e+10,other,57727,space,0.99973,1,0,1.00000,0,0,0,0,0,1,5,male,1,0,1,marketing,6,0.99962,146,liked,male,1,0,1,marketing,5,0.99899,22636,viewed,male,0,0,1,other,4,0.23077,3080,viewed,male,0,0,1,marketing,3,0.23077,44661,viewed,male,0,0,1,other,2,0.23077,3089,viewed,male,0,0,1,other,1,0.23077,0.73677
1,175348,1.514110e+12,1.515160e+12,6.309278e+10,6.204696e+10,forum,111913,space,0.99977,0,2,0.12336,0,0,0,1,0,0,146,male,1,0,1,marketing,6,0.99899,13659,commented,male,0,0,0,other,5,0.99756,146,commented,male,1,0,1,marketing,4,0.99899,22636,viewed,male,0,0,1,other,3,0.23077,3080,viewed,male,0,0,1,marketing,2,0.23077,44661,viewed,male,0,0,1,other,1,0.23077,0.78240
2,175353,1.514120e+12,1.515160e+12,6.309049e+10,6.204716e+10,forum,111913,space,0.99977,0,3,0.12336,0,0,0,1,0,0,146,male,1,0,1,marketing,6,0.99899,13659,commented,male,0,0,0,other,5,0.99756,13,commented,male,1,0,1,marketing,4,0.99895,146,commented,male,1,0,1,marketing,3,0.99899,22636,viewed,male,0,0,1,other,2,0.23077,3080,viewed,male,0,0,1,marketing,1,0.23077,0.82125
3,175371,1.514190e+12,1.514190e+12,6.301324e+10,6.301324e+10,other,9037,space,0.99949,0,0,0.00000,0,0,0,0,0,0,7,male,1,0,1,marketing,6,0.99886,22636,viewed,male,0,0,1,other,5,0.23077,3080,viewed,male,0,0,1,marketing,4,0.23077,44661,viewed,male,0,0,1,other,3,0.23077,3089,viewed,male,0,0,1,other,2,0.23077,3084,viewed,male,0,0,1,marketing,1,0.23077,0.61522
4,175372,1.514190e+12,1.514190e+12,6.301323e+10,6.301323e+10,other,9037,space,0.99949,0,0,0.00000,0,0,0,0,0,0,7,male,1,0,1,marketing,6,0.99886,22636,viewed,male,0,0,1,other,5,0.23077,3080,viewed,male,0,0,1,marketing,4,0.23077,44661,viewed,male,0,0,1,other,3,0.23077,3089,viewed,male,0,0,1,other,2,0.23077,3084,viewed,male,0,0,1,marketing,1,0.23077,0.61522


In [5]:
df.shape

(9418, 72)

# Creating synthetic features

In [6]:
df.posted_time /= 1000
df.updated_time /= 1000
df.age /= 1000
df.updated_age /= 1000

In [7]:
import datetime
import time

def get_date(secs):
  return datetime.datetime.fromtimestamp(secs).strftime("%A, %d/%m/%Y %H:%M:%S")

def get_date_separated(secs):
  date = datetime.datetime.fromtimestamp(secs)
  # return year, month, weekday, date(day),  hour, min, secs
  return int(date.strftime("%Y")), int(date.strftime("%m")), date.strftime("%A"), int(date.strftime("%d")), int(date.strftime("%H")), int(date.strftime("%M")), int(date.strftime("%S"))

In [8]:
# one minute = 60 secs
one_min = 60
one_hour = 60 * one_min

# define random values intervals
peak_interval_short = (10 * one_min, one_hour)
nonpeak_interval_short = (30 * one_min, 2 * one_hour)

peak_interval_middle = (10 * one_min, 2 * one_hour)
nonpeak_interval_middle = (30 * one_min, 4 * one_hour)

peak_interval_long = (10 * one_min, 4 * one_hour)
nonpeak_interval_long = (30 * one_min, 8 * one_hour)


# define start of peak time for each day

day2peak = {
    "Monday" : 8,
    "Tuesday" : 13,
    "Wednesday" : 14,
    "Thursday" : 9,
    "Friday" : 14
}

In [9]:
def is_peak(secs):
  date = get_date_separated(secs)
  weekday = date[2]

  # take start of the peak time from the date in seconds
  start = day2peak[weekday] * one_hour
  # define end as 3 hours after start
  end = start + 3 * one_hour
  
  # define current time in terms of seconds - seconds in hours + seconds in minutes and seconds
  current_secs = (one_hour * date[4]) + (one_min * date[5]) + date[6]

  return start <= current_secs  <= end

def in_working_time(secs):
  date = get_date_separated(secs)

  # define start and end of the working day, i.e 8 AM and 18 PM
  business_time_start = 8 * one_hour
  business_time_end = 18 * one_hour

  # define current time in terms of seconds - seconds in hours + seconds in minutes and seconds
  current_secs = (one_hour * date[4]) + (one_min * date[5]) + date[6]

  return business_time_start < current_secs < business_time_end

def is_weekend(secs):
  date = get_date_separated(secs)

  weekday = date[2]
  if weekday == 'Saturday' or weekday == 'Sunday':
    return True

In [10]:
import random

def create_effective_age(posted_time, peak, nonpeak):
  effective_ages = []


  for t in posted_time:


    if is_weekend(t):
      effective_age = random.randint(nonpeak[0], nonpeak[1]) * 2
      effective_ages.append(effective_age)
      continue

    if is_peak(t):
      effective_age = random.randint(peak[0], peak[1])
    else:
      effective_age = random.randint(nonpeak[0], nonpeak[1])

    if not in_working_time(t + effective_age):
      effective_age *= 2
  

    effective_ages.append(effective_age)

  return pd.Series(effective_ages)

In [11]:
df["effective_age_long"] = create_effective_age(df.posted_time, peak_interval_long, nonpeak_interval_long)

In [12]:
weekday2number = {"Monday" : 1,
                  "Tuesday" : 2,
                  "Wednesday" : 3,
                  "Thursday" : 4,
                  "Friday" : 5,
                  "Saturday" : 6,
                  "Sunday" : 7}


def create_weekday(posted_time):

  weekdays = []

  for t in posted_time:
    weekdays.append(weekday2number[get_date_separated(t)[2]])

  return weekdays

def create_daytime(posted_time):

  daytimes = []

  for t in posted_time:
    daytimes.append(get_date_separated(t)[4])

  return daytimes

In [13]:
posted_time = df.posted_time

df["weekday"] =  create_weekday(posted_time)
df["daytime"] =  create_daytime(posted_time)

# Data preprocessing

In [14]:
redundant_cols =  "id, updated_time, age, updated_age, owner_id, reactivity, poster_id, poster_is_lead, participant1_is_lead, participant2_is_lead, \
participant3_is_lead, participant4_is_lead, participant5_is_lead, poster_order, participant1_order, participant2_order, \
participant3_order, participant4_order, participant5_order, participant1_id, participant2_id, participant3_id, \
participant4_id, participant5_id".split(", ")


df.drop(redundant_cols, axis=1, inplace=True)

df.poster_focus.replace('none', 'other', inplace=True)

# Feature columns

In [15]:
_NUMERIC_COLUMNS = ['posted_time', 'owner_influence', 'poster_influence', 'participant1_influence', 'participant2_influence',
                    'participant3_influence', 'participant4_influence', 'participant5_influence']

_BINARY_COLUMNS = ["is_mentions_me", "is_mentions_connections", "is_commented_by_me", "is_commented_by_connections", 
                   "is_liked_by_me", "is_liked_by_connections", "poster_is_employee", "poster_is_in_connections",
                    "participant1_is_employee", "participant1_is_in_connections",
                    "participant2_is_employee", "participant2_is_in_connections",
                    "participant3_is_employee", "participant3_is_in_connections",
                    "participant4_is_employee", "participant4_is_in_connections",
                    "participant5_is_employee", "participant5_is_in_connections"]

_GENDER_COLUMNS = ["poster_gender", "participant1_gender", "participant2_gender", "participant3_gender", "participant4_gender", "participant5_gender"]

real = {
    colname : feature_column.numeric_column(colname) \
          for colname in _NUMERIC_COLUMNS
}


In [16]:
# indicator columns for categorical features

sparse = dict()

app_type = feature_column.categorical_column_with_vocabulary_list(
    'app_type', df.app_type.unique())
app_type_1hot = feature_column.indicator_column(app_type)
sparse["app_type"] = app_type_1hot

owner_type = feature_column.categorical_column_with_vocabulary_list(
    'owner_type', df.owner_type.unique())
owner_type_1hot = feature_column.indicator_column(owner_type)
sparse["owner_type"] = owner_type_1hot

poster_focus = feature_column.categorical_column_with_vocabulary_list(
    'poster_focus', ['engineering', 'sales', 'marketing', 'management', 'financial', 'other'])
poster_focus_1hot = feature_column.indicator_column(poster_focus)
sparse["poster_focus"] = poster_focus_1hot


for col in _GENDER_COLUMNS:
  feature = feature_column.categorical_column_with_vocabulary_list(
      col, df[col].unique())
  feature_1hot = feature_column.indicator_column(feature)
  sparse[col] = feature_1hot

# functions to reduce code duplication
def participant_action(part_action):
    participant_action = feature_column.categorical_column_with_vocabulary_list(
        part_action, ['commented', 'liked', 'viewed'])
    return participant_action

def participant_focus(part_f):
    participant_focus = feature_column.categorical_column_with_vocabulary_list(
        part_f, ['engineering', 'sales', 'marketing', 'management', 'financial', 'other', 'none'])
    return participant_focus

participant1_action = participant_action("participant1_action")
participant2_action = participant_action("participant2_action")
participant3_action = participant_action("participant3_action")
participant4_action = participant_action("participant4_action")
participant5_action = participant_action("participant5_action")

participant1_focus = participant_focus("participant1_focus")
participant2_focus = participant_focus("participant2_focus")
participant3_focus = participant_focus("participant3_focus")
participant4_focus = participant_focus("participant4_focus")
participant5_focus = participant_focus("participant5_focus")

sparse["participant1_action"] = feature_column.indicator_column(participant1_action)
sparse["participant2_action"] = feature_column.indicator_column(participant2_action)
sparse["participant3_action"] = feature_column.indicator_column(participant3_action)
sparse["participant4_action"] = feature_column.indicator_column(participant4_action)
sparse["participant5_action"] = feature_column.indicator_column(participant5_action)

sparse["participant1_focus"] = feature_column.indicator_column(participant1_focus)
sparse["participant2_focus"] = feature_column.indicator_column(participant2_focus)
sparse["participant3_focus"] = feature_column.indicator_column(participant3_focus)
sparse["participant4_focus"] = feature_column.indicator_column(participant4_focus)
sparse["participant5_focus"] = feature_column.indicator_column(participant5_focus)

In [17]:
inputs = {
    colname : tf.keras.layers.Input(name=colname, shape=(), dtype='float32') \
          for colname in real.keys()
}

inputs.update({
    colname : tf.keras.layers.Input(name=colname, shape=(), dtype='string') \
          for colname in sparse.keys()
})

In [18]:
for col in _BINARY_COLUMNS:
  feature = feature_column.categorical_column_with_vocabulary_list(
      col, df[col].unique())
  feature_1hot = feature_column.indicator_column(feature)
  sparse[col] = feature_1hot

likes_num = feature_column.numeric_column("number_of_likes")
likes_num_buckets = feature_column.bucketized_column(likes_num, boundaries=[2, 5, 10, 20, 50, 100])
sparse["number_of_likes"] = likes_num_buckets

comments_num = feature_column.numeric_column("number_of_comments")
comments_num_buckets = feature_column.bucketized_column(comments_num, boundaries=[1, 2, 5, 10, 20, 50, 100])
sparse["number_of_comments"] = comments_num_buckets


age_boundaries = [30 * one_min, one_hour,  2 * one_hour, 3 * one_hour, 4 * one_hour, 24 * one_hour]

age = feature_column.numeric_column("effective_age_long")
age_buckets = feature_column.bucketized_column(age, boundaries=age_boundaries)
sparse["effective_age_long"] = age_buckets

daytime = feature_column.numeric_column("daytime")
daytime_buckets = feature_column.bucketized_column(daytime, boundaries=[8, 12, 16, 24])
sparse["daytime"] = daytime_buckets

weekday = feature_column.categorical_column_with_vocabulary_list(
    'weekday', df.weekday.unique())
weekday_1hot = feature_column.indicator_column(weekday)
sparse["weekday"] = weekday_1hot

inputs.update({
    colname : tf.keras.layers.Input(name=colname, shape=(), dtype='int64') \
          for colname in _BINARY_COLUMNS + ["number_of_likes", "number_of_comments", "effective_age_long", "daytime", "weekday"]
})

# hash_bucket_size=30 because there are 7 possible values in weekday and 4 in daytime, all possible combinations will be 28 ~ 30
weekday_x_daytime = feature_column.crossed_column([weekday, daytime_buckets], hash_bucket_size=30)
sparse["weekday_x_daytime"] = feature_column.indicator_column(weekday_x_daytime)

# 6 bins in likes and 7 in comments
likes_x_comments = feature_column.crossed_column([likes_num_buckets, comments_num_buckets], hash_bucket_size=45)
sparse["likes_x_comments"] = feature_column.indicator_column(likes_x_comments)

# 6 bins in likes, 3 in action and 7 in focus, 6*3*7=126~130
likes_x_participant1_focus_n_action = feature_column.crossed_column([likes_num_buckets, participant1_action, participant1_focus], hash_bucket_size=130)
sparse["likes_x_participant1_focus_n_action"] = feature_column.indicator_column(likes_x_participant1_focus_n_action)

likes_x_participant2_focus_n_action = feature_column.crossed_column([likes_num_buckets, participant2_action, participant2_focus], hash_bucket_size=130)
sparse["likes_x_participant2_focus_n_action"] = feature_column.indicator_column(likes_x_participant2_focus_n_action)

likes_x_participant3_focus_n_action = feature_column.crossed_column([likes_num_buckets, participant3_action, participant3_focus], hash_bucket_size=130)
sparse["likes_x_participant3_focus_n_action"] = feature_column.indicator_column(likes_x_participant3_focus_n_action)

likes_x_participant4_focus_n_action = feature_column.crossed_column([likes_num_buckets, participant4_action, participant4_focus], hash_bucket_size=130)
sparse["likes_x_participant4_focus_n_action"] = feature_column.indicator_column(likes_x_participant4_focus_n_action)

likes_x_participant5_focus_n_action = feature_column.crossed_column([likes_num_buckets, participant5_action, participant5_focus], hash_bucket_size=130)
sparse["likes_x_participant5_focus_n_action"] = feature_column.indicator_column(likes_x_participant5_focus_n_action)

# Modelling

In [19]:
opt = tf.keras.optimizers.Adam()

def wide_and_deep_classifier(inputs, linear_feature_columns, dnn_feature_columns, dnn_hidden_units):
    deep = tf.keras.layers.DenseFeatures(dnn_feature_columns)(inputs)
    for numnodes in dnn_hidden_units:
        deep = tf.keras.layers.Dense(numnodes, activation='relu')(deep)
    wide = tf.keras.layers.DenseFeatures(linear_feature_columns)(inputs)
    both = tf.keras.layers.concatenate([deep, wide])
    output = tf.keras.layers.Dense(1)(both)
    model = tf.keras.Model(inputs, output)

    model.compile(loss='mean_absolute_error',
                  optimizer= opt,
                  metrics=['mean_absolute_error'])
    return model

model = wide_and_deep_classifier(inputs, sparse.values(), real.values(), [256, 512, 256, 256, 128, 64, 32, 16])

#  Training & Evaluating

In [20]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    dataframe = dataframe.copy()
    labels = dataframe.pop('rank')
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    return ds

In [21]:
train, test = train_test_split(df, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)

train_ds = df_to_dataset(train)
val_ds = df_to_dataset(val, shuffle=False)
test_ds = df_to_dataset(test, shuffle=False)

In [ ]:
model.fit(train_ds,
          validation_data=val_ds,
          epochs=50)

In [23]:
results = model.evaluate(test_ds, batch_size=32)
print("Test loss: {}\nTest accuracy: {}".format(results[0], results[1]))

59/59 [==============================] - 1s 13ms/step - loss: 0.0256 - mean_absolute_error: 0.0256
Test loss: 0.025593334808945656
Test accuracy: 0.025593334808945656
